In [1]:
# !pip install langgraph

In [2]:
# !pip install langchain langchain-community langchain-openai langchain_experimental langchain_groq  -q

In [3]:
from langchain.tools import Tool
from langchain_experimental.tools import PythonREPLTool
from langchain.document_loaders import TextLoader
from langchain_groq import ChatGroq
from langchain.agents import initialize_agent, load_tools
from langchain.agents import AgentType
#from langchain.memory import ConversationBufferMemory
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
import os
import re


In [4]:
def add(kwargs):
    """Adds two numbers."""
    try:
        num1, num2 = int(kwargs["num1"]), int(kwargs["num2"])
        return num1 + num2
    except Exception as e:
        return f"Error: {str(e)}"

def multiply(kwargs):
    """Multiplies two numbers."""
    try:
        num1, num2 = int(kwargs["num1"]), int(kwargs["num2"])
        return num1 * num2
    except Exception as e:
        return f"Error: {str(e)}"

def divide(kwargs):
    """Divides two numbers."""
    try:
        num1, num2 = float(kwargs["num1"]), float(kwargs["num2"])
        return num1 / num2 if num2 != 0 else "Cannot divide by zero"
    except Exception as e:
        return f"Error: {str(e)}"

def subtract(kwargs):
    try:
        num1, num2 = int(kwargs["num1"]), int(kwargs["num2"])
        return num1 - num2
    except Exception as e:
        return f"Error: {str(e)}"

def power(kwargs):
    """Raises the first number to the power of the second number."""
    try:
        num1, num2 = int(kwargs["num1"]), int(kwargs["num2"])
        return num1 ** num2
    except Exception as e:
        return f"Error: {str(e)}"

def summarize(kwargs):
    """Summarizes the given text."""
    return " ".join(kwargs.split()[:10]) + "..."

def word_count(kwargs):
    """Counts the number of words in the given text."""
    return len(kwargs.split(' '))


In [5]:
add_tool = Tool(name="Addition", func=add, description="Adds two numbers.")
multiply_tool = Tool(name="Multiplication", func=multiply, description="Multiplies two numbers.")
divide_tool = Tool(name="Division", func=divide, description="Divides two numbers.")
subtract_tool = Tool(name="Subtraction", func=subtract, description="Subtracts two numbers.")
power_tool = Tool(name="Power", func=power, description="Raises the first number to the power of the second number.")
summarize_tool = Tool(name="Summarization", func=summarize, description="Summarizes a given text.")
word_count_tool = Tool(name="WordCount", func=word_count, description="Counts words in a given text.")

In [6]:
tools = [add_tool, multiply_tool, divide_tool, subtract_tool,power_tool,summarize_tool,word_count_tool]

llm = ChatGroq(model_name="gemma2-9b-it", temperature=0,api_key = "api")

In [7]:
memory = MemorySaver()

In [8]:
agent = create_react_agent(llm, tools, checkpointer=memory)

In [9]:
config = {"configurable": {"thread_id": "abc123"}}
for chunk in agent.stream(
    {"messages": [HumanMessage(content="hi im bob! and i live in sf")]}, config
):
    print(chunk)
    print("----")



{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_gryg', 'function': {'arguments': '{"__arg1":"hi im bob! and i live in sf"}', 'name': 'Summarization'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 90, 'prompt_tokens': 1628, 'total_tokens': 1718, 'completion_time': 0.163636364, 'prompt_time': 0.078925933, 'queue_time': 0.022207434999999998, 'total_time': 0.242562297}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-e8933176-5dcf-4cbb-989b-80104391535e-0', tool_calls=[{'name': 'Summarization', 'args': {'__arg1': 'hi im bob! and i live in sf'}, 'id': 'call_gryg', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1628, 'output_tokens': 90, 'total_tokens': 1718})]}}
----
{'tools': {'messages': [ToolMessage(content='hi im bob! and i live in sf...', name='Summarization', id='046df9e4-b75f-4db2-a3bf-ce99f2ae4530', tool_call_id='call_g

In [ ]:
# response = agent.run("Do you know what my name is?")
# print(response)

In [10]:
for chunk in agent.stream(
    {"messages": [HumanMessage(content="Do you know my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Yes, your name is Bob.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 1744, 'total_tokens': 1753, 'completion_time': 0.016363636, 'prompt_time': 0.093372615, 'queue_time': 0.024080724999999997, 'total_time': 0.109736251}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-af0df18e-585e-4cf6-9f39-80a63b295359-0', usage_metadata={'input_tokens': 1744, 'output_tokens': 9, 'total_tokens': 1753})]}}
----


In [11]:
for chunk in agent.stream(
    {"messages": [HumanMessage(content="Tell me my name")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 1765, 'total_tokens': 1772, 'completion_time': 0.012727273, 'prompt_time': 0.113476172, 'queue_time': 0.02614569600000001, 'total_time': 0.126203445}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-e97ae77c-548d-4c00-8b22-d3664d2668d6-0', usage_metadata={'input_tokens': 1765, 'output_tokens': 7, 'total_tokens': 1772})]}}
----


In [12]:
for chunk in agent.stream(
    {"messages": [HumanMessage(content="Count word from i love bangladesh so much")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_x4aj', 'function': {'arguments': '{"__arg1":"i love bangladesh so much"}', 'name': 'WordCount'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 1789, 'total_tokens': 1840, 'completion_time': 0.092727273, 'prompt_time': 0.123665566, 'queue_time': 0.025625551999999996, 'total_time': 0.216392839}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-3badeaa8-79b6-4105-aaec-da6636ff2c66-0', tool_calls=[{'name': 'WordCount', 'args': {'__arg1': 'i love bangladesh so much'}, 'id': 'call_x4aj', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1789, 'output_tokens': 51, 'total_tokens': 1840})]}}
----
{'tools': {'messages': [ToolMessage(content='5', name='WordCount', id='bc00821b-1798-482c-9650-e10e05e7ecde', tool_call_id='call_x4aj')]}}
----
{'agent': {'messages': [AIMessa